In [3]:
import pandas as pd
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [4]:
file_path = r"C:\Users\pis05408.PINNACLE\Desktop\Suraj\Email_Sentiment_Data\email_dump_6thSept.parquet"

In [7]:
data = pd.read_parquet(file_path)
data.shape

(28766, 12)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28766 entries, 0 to 28765
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            28766 non-null  object 
 1   from_email    28766 non-null  object 
 2   to_email      28766 non-null  object 
 3   cc_email      28766 non-null  object 
 4   subject       28766 non-null  object 
 5   date_ist      28766 non-null  object 
 6   time_ist      28766 non-null  object 
 7   projectid     28766 non-null  int64  
 8   cleaned_body  28766 non-null  object 
 9   label         28766 non-null  object 
 10  score         28766 non-null  float32
 11  run_time      28766 non-null  int64  
dtypes: float32(1), int64(2), object(9)
memory usage: 2.5+ MB


In [23]:
selected_columns = ['id', 'subject', 'cleaned_body', 'date_ist', 'label']
output_path = 'C:/Users/pis05408.PINNACLE/Desktop/Suraj/Email_Sentiment_Data/'

# Function to clean illegal characters from text
def clean_text(text):
    if isinstance(text, str):
        # Replace control characters (ASCII < 32) with an empty string
        return re.sub(r'[\x00-\x1F\x7F]', '', text)
    return text

# Apply the cleaning function to the 'subject' and 'body' columns
data['subject'] = data['subject'].apply(clean_text)
data['cleaned_body'] = data['cleaned_body'].apply(clean_text)

# Write the cleaned DataFrame to an Excel file
data[selected_columns].to_excel(output_path + 'output_data_06Sep.xlsx', index=False)

print(f"Data successfully written to {output_path + 'output_data_06Sep.xlsx'}")

Data successfully written to C:/Users/pis05408.PINNACLE/Desktop/Suraj/Email_Sentiment_Data/output_data_06Sep.xlsx


In [9]:
df = data
min_count_per_label = df['label'].value_counts().min()

In [10]:
# Define the number of datasets (5 in this case)
n_datasets = 5

# Calculate the samples per dataset
samples_per_dataset = min_count_per_label // n_datasets

In [26]:
# Create a list to store the 5 datasets
datasets = []

# Split the data into 5 equally distributed datasets based on labels
for i in range(n_datasets):
    dataset = df.groupby('label', group_keys=False).apply(lambda x: x.sample(min(len(x), samples_per_dataset)))
    datasets.append(dataset)


C:\Users\pis05408.PINNACLE\AppData\Local\Temp\ipykernel_32980\1714450394.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dataset = df.groupby('label', group_keys=False).apply(lambda x: x.sample(min(len(x), samples_per_dataset)))
C:\Users\pis05408.PINNACLE\AppData\Local\Temp\ipykernel_32980\1714450394.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dataset = df.groupby('label', group_keys=False).appl

In [31]:
import os
# Define the path where the file will be saved
output_directory = r"C:\Users\pis05408.PINNACLE\Documents\email"
output_file = os.path.join(output_directory, 'output_data_06Sep.xlsx')  # Full file path

# Ensure the directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [32]:
# Write the datasets into the Excel file, each dataset in a separate sheet
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    for i, dataset in enumerate(datasets):
        # Write each dataset to a separate sheet
        dataset[selected_columns].to_excel(writer, sheet_name=f'Dataset_{i+1}', index=False)